In [1]:
# --- PASO 1: CARGAR MODELO Y PREPROCESADORES EXISTENTES ---
print(">>> Cargando modelo y preprocesadores existentes...")

import pandas as pd
import numpy as np
import pickle
from scipy.sparse import hstack
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
import xgboost as xgb

# Cargar modelo entrenado
with open('models\modelo_diagnostico_v6_xgboost.pkl', 'rb') as f:
    modelo_existente = pickle.load(f)

# Cargar preprocesadores
with open('models\preprocesadores_v6.pkl', 'rb') as f:
    preprocesadores = pickle.load(f)
    tfidf_vectorizer = preprocesadores['tfidf_vectorizer']
    age_encoder = preprocesadores['age_encoder']
    gender_encoder = preprocesadores['gender_encoder']
    diagnosis_encoder = preprocesadores['diagnosis_encoder']

print(">>> Modelo y preprocesadores cargados exitosamente!")

>>> Cargando modelo y preprocesadores existentes...
>>> Modelo y preprocesadores cargados exitosamente!


In [2]:
# --- PASO 2: CARGAR Y PREPARAR NUEVOS DATOS ---
print(">>> Cargando nuevo dataset...")

# Cargar el nuevo dataset
df_nuevo = pd.read_csv('datos/Disease_symptom_and_patient_profile_dataset.csv')
print(f"Nuevo dataset cargado: {df_nuevo.shape[0]} filas, {df_nuevo.shape[1]} columnas")

# Mostrar primeras filas para entender la estructura
print("\nPrimeras filas del nuevo dataset:")
print(df_nuevo.head())

# Mostrar columnas disponibles
print(f"\nColumnas disponibles: {list(df_nuevo.columns)}")

>>> Cargando nuevo dataset...
Nuevo dataset cargado: 349 filas, 10 columnas

Primeras filas del nuevo dataset:
       Disease Fever Cough Fatigue Difficulty Breathing  Age  Gender  \
0    Influenza   Yes    No     Yes                  Yes   19  Female   
1  Common Cold    No   Yes     Yes                   No   25  Female   
2       Eczema    No   Yes     Yes                   No   25  Female   
3       Asthma   Yes   Yes      No                  Yes   25    Male   
4       Asthma   Yes   Yes      No                  Yes   25    Male   

  Blood Pressure Cholesterol Level Outcome Variable  
0            Low            Normal         Positive  
1         Normal            Normal         Negative  
2         Normal            Normal         Negative  
3         Normal            Normal         Positive  
4         Normal            Normal         Positive  

Columnas disponibles: ['Disease', 'Fever', 'Cough', 'Fatigue', 'Difficulty Breathing', 'Age', 'Gender', 'Blood Pressure', 'Choleste

In [3]:
# --- PASO 3: TRANSFORMAR NUEVOS DATOS AL FORMATO DEL MODELO ORIGINAL ---
print(">>> Transformando nuevos datos al formato compatible...")

# Crear texto de "findings" basado en los síntomas
def crear_findings_texto(row):
    symptoms = []
    if row['Fever'] == 'Yes':
        symptoms.append('fever')
    if row['Cough'] == 'Yes':
        symptoms.append('cough')
    if row['Fatigue'] == 'Yes':
        symptoms.append('fatigue')
    if row['Difficulty Breathing'] == 'Yes':
        symptoms.append('difficulty breathing')
    
    # Agregar información adicional
    if row['Blood Pressure'] == 'High':
        symptoms.append('hypertension')
    elif row['Blood Pressure'] == 'Low':
        symptoms.append('hypotension')
        
    if row['Cholesterol Level'] == 'High':
        symptoms.append('high cholesterol')
    elif row['Cholesterol Level'] == 'Low':
        symptoms.append('low cholesterol')
    
    # Crear texto descriptivo
    findings_text = f"Patient presents with {', '.join(symptoms)}. "
    findings_text += f"Age: {row['Age']} years old, Gender: {row['Gender']}, "
    findings_text += f"Blood pressure: {row['Blood Pressure']}, Cholesterol: {row['Cholesterol Level']}."
    
    return findings_text

# Aplicar transformación
df_nuevo['Findings'] = df_nuevo.apply(crear_findings_texto, axis=1)

# Mapear rangos de edad (adaptar al formato original)
def mapear_edad(edad):
    if edad <= 20:
        return "0-20"
    elif edad <= 30:
        return "21-30"
    elif edad <= 40:
        return "31-40"
    elif edad <= 50:
        return "41-50"
    elif edad <= 60:
        return "51-60"
    elif edad <= 70:
        return "61-70"
    else:
        return "71+"

df_nuevo['Age Range'] = df_nuevo['Age'].apply(mapear_edad)
df_nuevo['Patient Gender'] = df_nuevo['Gender']
df_nuevo['Diagnosis Category'] = df_nuevo['Disease']

print(">>> Datos transformados exitosamente!")
print(f"Ejemplo de texto generado:")
print(df_nuevo['Findings'].iloc[0])

>>> Transformando nuevos datos al formato compatible...
>>> Datos transformados exitosamente!
Ejemplo de texto generado:
Patient presents with fever, fatigue, difficulty breathing, hypotension. Age: 19 years old, Gender: Female, Blood pressure: Low, Cholesterol: Normal.


In [4]:
# --- PASO 4: PREPARAR DATOS PARA REENTRENAMIENTO ---
print(">>> Preparando datos para reentrenamiento...")

# Importar función de preprocesamiento
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

# Descargar recursos necesarios si no están disponibles
try:
    word_tokenize("test")
    stopwords.words('english')
    WordNetLemmatizer().lemmatize("test")
except (LookupError, OSError):
    print("Descargando recursos de NLTK...")
    nltk.download('punkt')
    nltk.download('stopwords')
    nltk.download('wordnet')
    nltk.download('punkt_tab')

def preprocess_text_advanced(text):
    if pd.isna(text) or text is None:
        return ''
    
    text = str(text).lower()
    
    # Remover patrones médicos específicos pero preservar información importante
    text = re.sub(r'\b(patient|enrollee|reviewer|medical|treatment)\b', '', text)
    text = re.sub(r'\b\d+[-/]\d+[-/]\d+\b', '', text)  # Fechas
    text = re.sub(r'\b\d{2,4}\b', '', text)  # Años/números grandes
    
    # Preservar términos médicos importantes
    medical_terms = ['hypertension', 'diabetes', 'cardiac', 'hepatitis', 'autism', 
                    'depression', 'anxiety', 'therapy', 'surgery', 'medication',
                    'fever', 'cough', 'fatigue', 'breathing', 'cholesterol']
    
    text = text.translate(str.maketrans('', '', string.punctuation))
    tokens = word_tokenize(text)
    
    # Lemmatización
    lemmatizer = WordNetLemmatizer()
    stop_words = set(stopwords.words('english'))
    
    tokens = [lemmatizer.lemmatize(word) for word in tokens 
             if (word not in stop_words and len(word) > 2) or word in medical_terms]
    
    return ' '.join(tokens)

# Preprocesar texto
df_nuevo['clean_findings'] = df_nuevo['Findings'].apply(preprocess_text_advanced)

# Filtrar datos válidos
df_nuevo_clean = df_nuevo.dropna(subset=['Diagnosis Category', 'Age Range', 'Patient Gender'])

print(f"Datos limpios: {df_nuevo_clean.shape[0]} filas")
print(f"Enfermedades únicas en nuevos datos: {df_nuevo_clean['Diagnosis Category'].nunique()}")

>>> Preparando datos para reentrenamiento...
Datos limpios: 349 filas
Enfermedades únicas en nuevos datos: 116


In [5]:
# --- PASO 5: COMBINAR CON DATOS ORIGINALES (OPCIONAL) ---
print(">>> ¿Deseas combinar con datos originales? (recomendado)")

# Cargar datos originales
try:
    df_original = pd.read_csv('datos/Independent_Medical_Reviews.csv')
    df_original = df_original.dropna(subset=['Diagnosis Category'])
    df_original['clean_findings'] = df_original['Findings'].fillna('').apply(preprocess_text_advanced)
    
    print(f"Datos originales: {df_original.shape[0]} filas")
    
    # Combinar datasets
    df_combinado = pd.concat([
        df_original[['clean_findings', 'Age Range', 'Patient Gender', 'Diagnosis Category']],
        df_nuevo_clean[['clean_findings', 'Age Range', 'Patient Gender', 'Diagnosis Category']]
    ], ignore_index=True)
    
    print(f"Dataset combinado: {df_combinado.shape[0]} filas")
    usar_combinado = True
    
except Exception as e:
    print(f"No se pudieron cargar datos originales: {e}")
    print("Usando solo nuevos datos...")
    df_combinado = df_nuevo_clean[['clean_findings', 'Age Range', 'Patient Gender', 'Diagnosis Category']]
    usar_combinado = False

print(f"Enfermedades totales únicas: {df_combinado['Diagnosis Category'].nunique()}")

>>> ¿Deseas combinar con datos originales? (recomendado)
Datos originales: 19186 filas
Dataset combinado: 19535 filas
Enfermedades totales únicas: 145


In [6]:
# --- PASO 6: ACTUALIZAR ENCODERS CON NUEVAS CLASES ---
print(">>> Actualizando encoders con nuevas clases...")

# Preparar datos
X_text = df_combinado['clean_findings']
X_cat = df_combinado[['Age Range', 'Patient Gender']]
y = df_combinado['Diagnosis Category']

# Actualizar encoders para incluir nuevas clases
print("Clases de edad originales:", list(age_encoder.classes_))
print("Clases de género originales:", list(gender_encoder.classes_))
print("Clases de diagnóstico originales:", len(diagnosis_encoder.classes_))

# Identificar nuevas clases
nuevas_edades = set(X_cat['Age Range'].unique()) - set(age_encoder.classes_)
nuevos_generos = set(X_cat['Patient Gender'].unique()) - set(gender_encoder.classes_)
nuevos_diagnosticos = set(y.unique()) - set(diagnosis_encoder.classes_)

print(f"Nuevas edades encontradas: {nuevas_edades}")
print(f"Nuevos géneros encontrados: {nuevos_generos}")
print(f"Nuevos diagnósticos encontrados: {len(nuevos_diagnosticos)} - {list(nuevos_diagnosticos)[:5]}...")

# Crear nuevos encoders que incluyan todas las clases
from sklearn.preprocessing import LabelEncoder

# Actualizar encoder de edad
if nuevas_edades:
    todas_edades = list(age_encoder.classes_) + list(nuevas_edades)
    age_encoder_nuevo = LabelEncoder()
    age_encoder_nuevo.fit(todas_edades)
else:
    age_encoder_nuevo = age_encoder

# Actualizar encoder de género
if nuevos_generos:
    todos_generos = list(gender_encoder.classes_) + list(nuevos_generos)
    gender_encoder_nuevo = LabelEncoder()
    gender_encoder_nuevo.fit(todos_generos)
else:
    gender_encoder_nuevo = gender_encoder

# Actualizar encoder de diagnóstico
if nuevos_diagnosticos:
    todos_diagnosticos = list(diagnosis_encoder.classes_) + list(nuevos_diagnosticos)
    diagnosis_encoder_nuevo = LabelEncoder()
    diagnosis_encoder_nuevo.fit(todos_diagnosticos)
else:
    diagnosis_encoder_nuevo = diagnosis_encoder

print(f"Clases finales - Edad: {len(age_encoder_nuevo.classes_)}, Género: {len(gender_encoder_nuevo.classes_)}, Diagnóstico: {len(diagnosis_encoder_nuevo.classes_)}")

>>> Actualizando encoders con nuevas clases...
Clases de edad originales: ['0-10', '11_20', '21-30', '31-40', '41-50', '51-64', '65+', 'Unknown']
Clases de género originales: ['Female', 'Male', 'Unknown']
Clases de diagnóstico originales: 30
Nuevas edades encontradas: {'0-20', '71+', nan, '61-70', '51-60'}
Nuevos géneros encontrados: {nan}
Nuevos diagnósticos encontrados: 116 - ['Tetanus', 'Sinusitis', 'Gastroenteritis', 'Conjunctivitis (Pink Eye)', "Crohn's Disease"]...
Clases finales - Edad: 13, Género: 4, Diagnóstico: 146


In [13]:
# --- PASO 7: REENTRENAR MODELO (CORREGIDO) ---
print(">>> Reentrenando modelo con nuevos datos...")

# Preparar características
X_cat_encoded = pd.DataFrame({
    'age_encoded': age_encoder_nuevo.transform(X_cat['Age Range']),
    'gender_encoded': gender_encoder_nuevo.transform(X_cat['Patient Gender'])
})

# Vectorizar texto (actualizar TF-IDF con nuevo vocabulario)
tfidf_nuevo = TfidfVectorizer(
    max_features=5000,
    ngram_range=(1, 3),
    min_df=2,
    max_df=0.95,
    sublinear_tf=True,
    stop_words='english'
)

X_tfidf = tfidf_nuevo.fit_transform(X_text)

# Combinar características
X_combined = hstack([X_tfidf, X_cat_encoded.values])

# Codificar target
y_encoded = diagnosis_encoder_nuevo.transform(y)

# Verificar clases con pocas muestras
from collections import Counter
class_counts = Counter(y_encoded)
min_samples = min(class_counts.values())
classes_with_few_samples = [cls for cls, count in class_counts.items() if count < 2]

print(f"Clases con menos de 2 muestras: {len(classes_with_few_samples)}")
print(f"Mínimo de muestras por clase: {min_samples}")

# FIX 1: División de datos mejorada
try:
    if min_samples >= 2:
        X_train, X_test, y_train, y_test = train_test_split(
            X_combined, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded
        )
        print("División estratificada aplicada")
    else:
        X_train, X_test, y_train, y_test = train_test_split(
            X_combined, y_encoded, test_size=0.2, random_state=42
        )
        print("División aleatoria aplicada (algunas clases tienen muy pocas muestras)")
except Exception as e:
    print(f"Error en la división de datos: {e}")
    print("Aplicando división simple...")
    X_train, X_test, y_train, y_test = train_test_split(
        X_combined, y_encoded, test_size=0.2, random_state=42
    )

# FIX 2: Manejo robusto de etiquetas consecutivas
print(">>> Creando mapeo de etiquetas...")

# Obtener todas las etiquetas únicas del conjunto completo
unique_labels = np.unique(y_encoded)
print(f"Etiquetas únicas encontradas: {len(unique_labels)}")
print(f"Rango de etiquetas originales: {min(unique_labels)} a {max(unique_labels)}")

# Crear mapeo bidireccional
label_mapping = {old_label: new_label for new_label, old_label in enumerate(unique_labels)}
reverse_mapping = {new_label: old_label for old_label, new_label in label_mapping.items()}

print(f"Mapeo creado: {len(label_mapping)} clases")

# FIX 3: Aplicar mapeo de forma segura (CORREGIDO)
print(">>> Creando mapeo consecutivo correcto...")

try:
    # Obtener todas las etiquetas únicas del conjunto de entrenamiento
    unique_train_labels = np.unique(y_train)
    
    print(f"Etiquetas únicas en entrenamiento: {len(unique_train_labels)}")
    print(f"Primer mapeo - etiquetas encontradas: {unique_train_labels[:10]}...")
    
    # Crear mapeo consecutivo SOLO para las clases presentes en entrenamiento
    label_mapping = {old_label: new_label for new_label, old_label in enumerate(unique_train_labels)}
    reverse_mapping = {new_label: old_label for old_label, new_label in label_mapping.items()}
    
    # Aplicar mapeo
    y_train_consecutive = np.array([label_mapping[label] for label in y_train])
    
    # Para el conjunto de prueba, mapear solo las etiquetas que existen en entrenamiento
    y_test_consecutive = []
    valid_test_indices = []
    
    for i, label in enumerate(y_test):
        if label in label_mapping:
            y_test_consecutive.append(label_mapping[label])
            valid_test_indices.append(i)
        # Si la etiqueta no está en entrenamiento, la excluimos de la evaluación
    
    y_test_consecutive = np.array(y_test_consecutive)
    valid_test_indices = np.array(valid_test_indices)
    
    # Filtrar X_test para mantener solo las muestras válidas
    X_test_filtered = X_test[valid_test_indices]
    
    print(f"Entrenamiento: {len(y_train_consecutive)} muestras con {len(np.unique(y_train_consecutive))} clases")
    print(f"Prueba: {len(y_test_consecutive)} muestras válidas (de {len(y_test)} originales)")
    print(f"Clases consecutivas: {np.min(y_train_consecutive)} a {np.max(y_train_consecutive)}")
    
    # Verificar que ahora SÍ son consecutivas
    expected_consecutive = np.arange(len(unique_train_labels))
    unique_train_consecutive = np.unique(y_train_consecutive)
    
    if np.array_equal(unique_train_consecutive, expected_consecutive):
        print("✅ Etiquetas de entrenamiento son perfectamente consecutivas")
    else:
        print("❌ Aún hay problemas con las etiquetas consecutivas")
        print(f"Esperado: {expected_consecutive}")
        print(f"Obtenido: {unique_train_consecutive}")
        
except Exception as e:
    print(f"Error en mapeo de etiquetas: {e}")
    # Fallback más simple
    y_train_consecutive = y_train
    y_test_consecutive = y_test
    X_test_filtered = X_test
    label_mapping = {i: i for i in range(len(np.unique(y_train)))}
    reverse_mapping = label_mapping.copy()

# FIX 4: Crear modelo con parámetros robustos
print(">>> Creando modelo XGBoost...")

modelo_reentrenado = xgb.XGBClassifier(
    objective='multi:softprob',
    n_estimators=150,  # Reducir aún más
    max_depth=5,       # Profundidad más conservadora
    learning_rate=0.1,
    subsample=0.8,
    colsample_bytree=0.8,
    reg_alpha=0.1,
    reg_lambda=1.5,
    random_state=42,
    n_jobs=-1,
    eval_metric='mlogloss',
    verbosity=0        # Reducir verbosidad
)

# FIX 5: Entrenamiento con verificación estricta
print(">>> Entrenando modelo...")
try:
    # Verificaciones finales antes del entrenamiento
    print(f"Dimensiones X_train: {X_train.shape}")
    print(f"Dimensiones y_train: {y_train_consecutive.shape}")
    print(f"Clases únicas en y_train_consecutive: {len(np.unique(y_train_consecutive))}")
    print(f"Rango de clases: {np.min(y_train_consecutive)} a {np.max(y_train_consecutive)}")
    
    # Verificación estricta de consecutividad
    unique_classes = np.unique(y_train_consecutive)
    expected_classes = np.arange(len(unique_classes))
    
    if not np.array_equal(unique_classes, expected_classes):
        print("❌ Las clases TODAVÍA no son consecutivas. Corrigiendo...")
        
        # Crear un mapeo forzado consecutivo
        final_mapping = {old: new for new, old in enumerate(unique_classes)}
        y_train_consecutive = np.array([final_mapping[label] for label in y_train_consecutive])
        y_test_consecutive = np.array([final_mapping[label] if label in final_mapping else 0 
                                     for label in y_test_consecutive])
        
        # Actualizar mapeos
        for old_label, temp_label in label_mapping.items():
            if temp_label in final_mapping:
                label_mapping[old_label] = final_mapping[temp_label]
        
        reverse_mapping = {new_label: old_label for old_label, new_label in label_mapping.items()}
        
        print(f"✅ Clases forzadas a ser consecutivas: 0 a {len(unique_classes)-1}")
    
    # Entrenar modelo
    print("Iniciando entrenamiento...")
    modelo_reentrenado.fit(X_train, y_train_consecutive)
    print("✅ Entrenamiento completado exitosamente!")
    
except Exception as e:
    print(f"❌ Error durante el entrenamiento: {e}")
    print("Intentando entrenamiento con modelo ultra-simplificado...")
    
    # Modelo ultra-simplificado como último recurso
    modelo_reentrenado = xgb.XGBClassifier(
        objective='multi:softprob',
        n_estimators=50,
        max_depth=3,
        learning_rate=0.1,
        random_state=42,
        n_jobs=1  # Un solo hilo para evitar problemas
    )
    
    try:
        modelo_reentrenado.fit(X_train, y_train_consecutive)
        print("✅ Entrenamiento ultra-simplificado completado!")
    except Exception as e2:
        print(f"❌ Error crítico: {e2}")
        print("No se pudo entrenar el modelo. Revisa los datos.")

>>> Reentrenando modelo con nuevos datos...
Clases con menos de 2 muestras: 61
Mínimo de muestras por clase: 1
División aleatoria aplicada (algunas clases tienen muy pocas muestras)
>>> Creando mapeo de etiquetas...
Etiquetas únicas encontradas: 145
Rango de etiquetas originales: 0 a 144
Mapeo creado: 145 clases
>>> Creando mapeo consecutivo correcto...
Etiquetas únicas en entrenamiento: 130
Primer mapeo - etiquetas encontradas: [0 1 2 3 4 5 6 7 8 9]...
Entrenamiento: 15628 muestras con 130 clases
Prueba: 3892 muestras válidas (de 3907 originales)
Clases consecutivas: 0 a 129
✅ Etiquetas de entrenamiento son perfectamente consecutivas
>>> Creando modelo XGBoost...
>>> Entrenando modelo...
Dimensiones X_train: (15628, 5002)
Dimensiones y_train: (15628,)
Clases únicas en y_train_consecutive: 130
Rango de clases: 0 a 129
Iniciando entrenamiento...
✅ Entrenamiento completado exitosamente!


In [14]:
# FIX 6: Evaluación con datos filtrados
print(">>> Evaluando modelo...")
try:
    if len(y_test_consecutive) > 0:
        y_pred = modelo_reentrenado.predict(X_test_filtered)
        accuracy = accuracy_score(y_test_consecutive, y_pred)
        
        print(f"\n>>> RESULTADOS DEL REENTRENAMIENTO <<<")
        print(f"Precisión del modelo reentrenado: {accuracy * 100:.2f}%")
        print(f"Datos de entrenamiento: {X_train.shape[0]} muestras")
        print(f"Datos de prueba evaluados: {len(y_test_consecutive)} muestras")
        print(f"Datos de prueba excluidos: {len(y_test) - len(y_test_consecutive)} muestras")
        print(f"Nuevas clases de diagnóstico añadidas: {len(nuevos_diagnosticos)}")
        print(f"Total de enfermedades que puede predecir: {len(diagnosis_encoder_nuevo.classes_)}")
        
        # Verificar predicciones
        unique_predictions = np.unique(y_pred)
        print(f"Predicciones únicas generadas: {len(unique_predictions)}")
        
        # Mostrar distribución de confianza
        y_pred_proba = modelo_reentrenado.predict_proba(X_test_filtered)
        max_probas = np.max(y_pred_proba, axis=1)
        
        print(f"\nDistribución de confianza:")
        print(f"Confianza promedio: {np.mean(max_probas)*100:.2f}%")
        print(f"Confianza mínima: {np.min(max_probas)*100:.2f}%")
        print(f"Confianza máxima: {np.max(max_probas)*100:.2f}%")
        
        # Predicciones con alta confianza (>70%)
        high_confidence = np.sum(max_probas > 0.7)
        print(f"Predicciones con >70% confianza: {high_confidence}/{len(max_probas)} ({high_confidence/len(max_probas)*100:.1f}%)")
        
    else:
        print("❌ No hay datos de prueba válidos para evaluar")
        accuracy = 0.0
        
except Exception as e:
    print(f"❌ Error durante la evaluación: {e}")
    accuracy = 0.0

print(">>> Proceso de reentrenamiento finalizado!")

>>> Evaluando modelo...

>>> RESULTADOS DEL REENTRENAMIENTO <<<
Precisión del modelo reentrenado: 75.18%
Datos de entrenamiento: 15628 muestras
Datos de prueba evaluados: 3892 muestras
Datos de prueba excluidos: 15 muestras
Nuevas clases de diagnóstico añadidas: 116
Total de enfermedades que puede predecir: 146
Predicciones únicas generadas: 52

Distribución de confianza:
Confianza promedio: 76.04%
Confianza mínima: 6.18%
Confianza máxima: 99.93%
Predicciones con >70% confianza: 2579/3892 (66.3%)
>>> Proceso de reentrenamiento finalizado!


In [15]:
# --- PASO 8: GUARDAR MODELO REENTRENADO ---
print(">>> Guardando modelo reentrenado...")

try:
    # Guardar modelo reentrenado
    with open('models/modelo_diagnostico_v8_reentrenado.pkl', 'wb') as f:
        pickle.dump(modelo_reentrenado, f)

    # Guardar preprocesadores actualizados y mappings
    with open('models/preprocesadores_v8_reentrenado.pkl', 'wb') as f:
        pickle.dump({
            'tfidf_vectorizer': tfidf_nuevo,
            'age_encoder': age_encoder_nuevo,
            'gender_encoder': gender_encoder_nuevo,
            'diagnosis_encoder': diagnosis_encoder_nuevo,
            'label_mapping': label_mapping,
            'reverse_mapping': reverse_mapping,
            'unique_labels': unique_labels
        }, f)

    print("✅ Modelo v8 guardado exitosamente!")
    print("Archivos creados:")
    print("- modelo_diagnostico_v8_reentrenado.pkl")
    print("- preprocesadores_v8_reentrenado.pkl")
    
except Exception as e:
    print(f"❌ Error guardando modelo: {e}")

>>> Guardando modelo reentrenado...
✅ Modelo v8 guardado exitosamente!
Archivos creados:
- modelo_diagnostico_v8_reentrenado.pkl
- preprocesadores_v8_reentrenado.pkl


In [16]:
# --- PASO 9: FUNCIÓN DE PREDICCIÓN ACTUALIZADA ---
def predecir_diagnostico_v8(texto_findings, edad_rango, genero_paciente):
    """
    Función de predicción con modelo v8 reentrenado
    """
    try:
        # Preprocesar texto
        texto_limpio = preprocess_text_advanced(texto_findings)
        
        # Vectorizar
        texto_tfidf = tfidf_nuevo.transform([texto_limpio])
        
        # Validar y codificar categóricas
        if edad_rango not in age_encoder_nuevo.classes_:
            print(f"Advertencia: Edad '{edad_rango}' no reconocida. Usando '31-40'.")
            edad_rango = "31-40"  # Valor por defecto común
            
        if genero_paciente not in gender_encoder_nuevo.classes_:
            print(f"Advertencia: Género '{genero_paciente}' no reconocido. Usando 'Male'.")
            genero_paciente = "Male"  # Valor por defecto
            
        edad_encoded = age_encoder_nuevo.transform([edad_rango])
        genero_encoded = gender_encoder_nuevo.transform([genero_paciente])
        
        # Combinar características
        caracteristicas_categoricas = np.array([[edad_encoded[0], genero_encoded[0]]])
        caracteristicas_combinadas = hstack([texto_tfidf, caracteristicas_categoricas])
        
        # Predicción con etiquetas consecutivas
        prediccion_consecutiva = modelo_reentrenado.predict(caracteristicas_combinadas)
        probabilidades = modelo_reentrenado.predict_proba(caracteristicas_combinadas)
        
        # Convertir de etiquetas consecutivas a originales
        prediccion_original = reverse_mapping[prediccion_consecutiva[0]]
        
        # Decodificar a nombre de enfermedad
        categoria_predicha = diagnosis_encoder_nuevo.inverse_transform([prediccion_original])[0]
        confianza = max(probabilidades[0]) * 100
        
        # Top 5 predicciones
        top_indices = np.argsort(probabilidades[0])[::-1][:5]
        top_categorias = []
        for i in top_indices:
            # Convertir índice consecutivo a etiqueta original
            etiqueta_original = reverse_mapping[i]
            categoria = diagnosis_encoder_nuevo.inverse_transform([etiqueta_original])[0]
            prob = probabilidades[0][i] * 100
            top_categorias.append({
                "disease": categoria,
                "probability": round(prob, 2)
            })
            
        return {
            'success': True,
            'main_diagnosis': categoria_predicha,
            'confidence': round(confianza, 2),
            'top_predictions': top_categorias,
            'processed_text': texto_limpio,
            'model_version': 'v8_reentrenado'
        }
        
    except Exception as e:
        return {
            'success': False,
            'error': str(e),
            'model_version': 'v8_reentrenado'
        }

print(">>> Función de predicción v8 creada exitosamente!")

>>> Función de predicción v8 creada exitosamente!


In [17]:
# --- PASO 10: PRUEBAS DEL MODELO REENTRENADO ---
print("\n>>> Probando modelo v8 reentrenado...")

# Casos de prueba específicos del nuevo dataset
casos_prueba_v8 = [
    {
        "nombre": "Asma",
        "symptoms": "Patient presents with cough, difficulty breathing, wheezing",
        "age_range": "21-30",
        "gender": "Male"
    },
    {
        "nombre": "Diabetes", 
        "symptoms": "Patient reports fatigue, excessive thirst, frequent urination",
        "age_range": "41-50",
        "gender": "Female"
    },
    {
        "nombre": "Influenza",
        "symptoms": "Patient presents with fever, cough, body aches, fatigue",
        "age_range": "21-30",
        "gender": "Female"
    },
    {
        "nombre": "Migraña",
        "symptoms": "Patient reports severe headache, sensitivity to light, nausea",
        "age_range": "31-40",
        "gender": "Female"
    },
    {
        "nombre": "Eczema",
        "symptoms": "Patient has skin rash, itching, dry skin patches",
        "age_range": "21-30",
        "gender": "Female"
    }
]

print("\n=== PRUEBAS DEL MODELO v8 ===")
aciertos = 0
total_casos = len(casos_prueba_v8)

for i, caso in enumerate(casos_prueba_v8, 1):
    print(f"\n--- CASO {i}: {caso['nombre']} ---")
    
    resultado = predecir_diagnostico_v8(
        caso['symptoms'],
        caso['age_range'], 
        caso['gender']
    )
    
    if resultado['success']:
        print(f"Predicción: {resultado['main_diagnosis']}")
        print(f"Confianza: {resultado['confidence']}%")
        print("Top 3 predicciones:")
        for j, pred in enumerate(resultado['top_predictions'][:3], 1):
            print(f"  {j}. {pred['disease']}: {pred['probability']}%")
        
        # Verificar si la predicción es correcta
        esperado = caso['nombre'].lower()
        prediccion = resultado['main_diagnosis'].lower()
        
        # Buscar en predicción principal y top 3
        encontrado = (esperado in prediccion or 
                     any(esperado in pred['disease'].lower() 
                         for pred in resultado['top_predictions'][:3]))
        
        if encontrado:
            print("✅ Predicción CORRECTA")
            aciertos += 1
        else:
            print("❌ Predicción incorrecta")
    else:
        print(f"❌ Error: {resultado['error']}")

# Calcular precisión en casos de prueba
precision_casos = (aciertos / total_casos) * 100
print(f"\n📊 RESUMEN DE PRUEBAS:")
print(f"Casos correctos: {aciertos}/{total_casos}")
print(f"Precisión en casos de prueba: {precision_casos:.1f}%")

print(f"\n>>> ¡Modelo v8 reentrenado completado!")
print(f"📊 Precisión general: {accuracy*100:.2f}%")
print(f"🎯 Precisión en pruebas específicas: {precision_casos:.1f}%")
print(f"🏥 Total enfermedades: {len(diagnosis_encoder_nuevo.classes_)}")
print(f"🆕 Nuevas enfermedades añadidas: {len(nuevos_diagnosticos)}")


>>> Probando modelo v8 reentrenado...

=== PRUEBAS DEL MODELO v8 ===

--- CASO 1: Asma ---
Predicción: Orthopedic/ Musculoskeletal
Confianza: 17.809999465942383%
Top 3 predicciones:
  1. Orthopedic/ Musculoskeletal: 17.809999465942383%
  2. Central Nervous System/ Neuromuscular: 17.760000228881836%
  3. Mental: 9.520000457763672%
❌ Predicción incorrecta

--- CASO 2: Diabetes ---
Predicción: Central Nervous System/ Neuromuscular
Confianza: 19.3700008392334%
Top 3 predicciones:
  1. Central Nervous System/ Neuromuscular: 19.3700008392334%
  2. Orthopedic/ Musculoskeletal: 15.5600004196167%
  3. Cancer: 8.210000038146973%
❌ Predicción incorrecta

--- CASO 3: Influenza ---
Predicción: Central Nervous System/ Neuromuscular
Confianza: 15.270000457763672%
Top 3 predicciones:
  1. Central Nervous System/ Neuromuscular: 15.270000457763672%
  2. Orthopedic/ Musculoskeletal: 14.529999732971191%
  3. Mental: 8.180000305175781%
❌ Predicción incorrecta

--- CASO 4: Migraña ---
Predicción: Central N